In [ ]:
Using LLM to continue the analysis
Testing the predictive power of changes in the Offensive Raing and Defensive Raing individually to determine which side of the ball has a stonger statistical impact on becoming a contender

Running 2 separate Logistic Regression models



In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

# --- 1. CONFIGURATION (Using the statistically appropriate names) ---
FILE_NAME = 'Team Records.csv' 
TEAM_COL = 'Team'
SEASON_COL = 'Season' 
REL_ORTG_COL = 'Rel_ORtg'  # Used for regression
REL_DRTG_COL = 'Rel_DRtg'  # Used for regression
PLAYOFF_COL = 'Playoffs'

# --- 2. DATA CLEANING & PREP ---
df = pd.read_csv(FILE_NAME) 

# Clean NaNs
df.dropna(subset=[REL_ORTG_COL, REL_DRTG_COL], inplace=True)

# Create Year Columns
df['StartYear'] = df[SEASON_COL].str[:4].astype('Int64')
df['FinishYear'] = df['StartYear'] + 1
SEASON_FILTER_COL = 'FinishYear'

# Filter to 30-team era
df_modern = df[df[SEASON_FILTER_COL] >= 2005].copy()

# Create IsContender Variable
contender_outcomes = ['Finals Winner', 'Finals Loser', 'Conf. Finals'] # Use key strings from the mapping function

def map_is_contender(result):
    result_str = str(result)
    # Check for the key phrases defined in the mapping function
    if 'Won Finals' in result_str or 'Lost Finals' in result_str or 'Conf. Finals' in result_str:
        return 1
    return 0

df_modern['IsContender'] = df_modern[PLAYOFF_COL].apply(map_is_contender) 

# --- 3. MODEL A: Offensive Impact (Rel_ORtg) ---

X_off = sm.add_constant(df_modern[REL_ORTG_COL])
Y = df_modern['IsContender']

model_off = sm.Logit(Y, X_off)
result_off = model_off.fit(disp=False) 

print("### 🏀 Model A: Offense (Rel_ORtg) Impact")
print(result_off.summary().as_text())

# --- 4. MODEL B: Defensive Impact (Rel_DRtg) ---

X_def = sm.add_constant(df_modern[REL_DRTG_COL])
Y = df_modern['IsContender']

model_def = sm.Logit(Y, X_def)
result_def = model_def.fit(disp=False)

print("\n### 🛡️ Model B: Defense (Rel_DRtg) Impact")
print(result_def.summary().as_text())

### 🏀 Model A: Offense (Rel_ORtg) Impact
                           Logit Regression Results                           
Dep. Variable:            IsContender   No. Observations:                  420
Model:                          Logit   Df Residuals:                      418
Method:                           MLE   Df Model:                            1
Date:                Tue, 25 Nov 2025   Pseudo R-squ.:                  0.2022
Time:                        14:35:02   Log-Likelihood:                -125.46
converged:                       True   LL-Null:                       -157.27
Covariance Type:            nonrobust   LLR p-value:                 1.519e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.5438      0.223    -11.390      0.000      -2.982      -2.106
Rel_ORtg       0.4132      0.060      6.915      0.000       0.296       0.530

### 🛡️ Mod

In [20]:
# Defense is a slightly stronger independent predictor of reaching the conference finals
# Predictive Power (R^2): Model A: 0.2022, Model B: 0.2161
# Defense explains more variability in contender status
# Coefficients: Model A: +0.4132, Model B: -0.4540
# Defence has a slightly larger magnitude of impact

# 1. Predictive Strength (Pseudo R-squared)The $\text{Pseudo R-squared}$ is your best metric for overall predictive power.
# Defense (Model B) has a $\text{Pseudo R-squared}$ of $0.2161$ ($\mathbf{21.61\%}$).
# Offense (Model A) has a $\text{Pseudo R-squared}$ of $0.2022$ ($\mathbf{20.22\%}$).

# Defense explains about $1.4$ percentage points more of the variability in contender status than Offense does when tested as a single predictor.

# 2. Magnitude of Impact (Coefficients)The coefficients show the change in log-odds for a 1-point movement in the rating.
# Offense ($\text{Rel\_ORtg}$):$\text{Coefficient} = +0.4132$
# Interpretation: A 1-point increase in a team's Offensive Rating (relative to the league average) increases the odds of being a contender by about 1.51 times.
# Defense ($\text{Rel\_DRtg}$):$\text{Coefficient} = -0.4540$ (Note: A negative coefficient is good for defense, as a lower rating is better).
# Interpretation: A 1-point decrease in a team's Defensive Rating (relative to the league average) increases the odds of being a contender by about 1.57 times ($e^{0.4540}$)

# ConclusionThe data supports the basketball maxim: Defense travels in the playoffs. 
# While both offensive and defensive efficiency are highly significant predictors of success, a single unit of improvement in Defensive Rating provides a slightly larger boost to the probability of reaching the Conference Finals than the equivalent unit of improvement in Offensive Rating.

In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import accuracy_score

# --- 1. CONFIGURATION ---
FILE_NAME = 'Team Records.csv' 
TEAM_COL = 'Team'
SEASON_COL = 'Season' 
REL_ORTG_COL = 'Rel_ORtg' 
REL_DRTG_COL = 'Rel_DRtg'
PLAYOFF_COL = 'Playoffs'

# --- 2. DATA PREP: LOAD, CLEAN, and FEATURE ENGINEERING ---

def map_is_contender(result):
    result_str = str(result)
    # Contender = Conference Finals or better
    if 'Won Finals' in result_str or 'Lost Finals' in result_str or 'Conf. Finals' in result_str:
        return 1
    return 0

# Load data 
df = pd.read_csv(FILE_NAME) 

# Clean NaNs and filter to 30-team era (2005 onwards)
df.dropna(subset=[REL_ORTG_COL, REL_DRTG_COL], inplace=True)
df['StartYear'] = df[SEASON_COL].str[:4].astype('Int64')
df['FinishYear'] = df['StartYear'] + 1
df_modern = df[df['FinishYear'] >= 2005].copy()

# Create Target Variable (Y)
df_modern['IsContender'] = df_modern[PLAYOFF_COL].apply(map_is_contender) 

# --- Feature Engineering ---
# 1. OffRank & DefRank (REQUIRED FOR CONTENDER SCORE)
df_modern['OffRank'] = df_modern.groupby('FinishYear')[REL_ORTG_COL].rank(method='min', ascending=False).astype('Int64')
df_modern['DefRank'] = df_modern.groupby('FinishYear')[REL_DRTG_COL].rank(method='min', ascending=True).astype('Int64')

# 2. ContenderScore (Based on Rank)
df_modern['ContenderScore'] = 10 * ( (30 - df_modern['OffRank']) + (30 - df_modern['DefRank']) ) / 60

# 3. EliteBonus (Non-linear Feature - The Fix)
ELITE_THRESHOLD = 5 
df_modern['EliteBonus'] = np.where(
    (df_modern['OffRank'] <= ELITE_THRESHOLD) & (df_modern['DefRank'] <= ELITE_THRESHOLD), 
    1, 
    0
)

# Define Y for all models
Y = df_modern['IsContender']

print("---")
print("✅ Data Prep Complete. Running All Models:")

# ==============================================================================
# HYPOTHESIS 1: Original Hypothesis (ContenderScore Validity)
# ==============================================================================
# X1 = ContenderScore only (Linear Model)
X1 = sm.add_constant(df_modern['ContenderScore'].astype(float))
model1 = sm.Logit(Y, X1).fit(disp=False)
print("\n## 1. Original Hypothesis: ContenderScore Validity")
print(model1.summary().as_text())

# ==============================================================================
# HYPOTHESIS 2: Offense vs. Defense Impact
# ==============================================================================

print("\n## 2. Offense vs. Defense Impact (Comparing Pseudo R-squared)")

# Model A: Offensive Impact
X2_off = sm.add_constant(df_modern[REL_ORTG_COL].astype(float))
model2_off = sm.Logit(Y, X2_off).fit(disp=False)
print("### 🏀 Model A: Offense (Rel_ORtg) - Pseudo R-squared:")
print(f"R-squared: {model2_off.prsquared:.4f}")

# Model B: Defensive Impact
X2_def = sm.add_constant(df_modern[REL_DRTG_COL].astype(float))
model2_def = sm.Logit(Y, X2_def).fit(disp=False)
print("### 🛡️ Model B: Defense (Rel_DRtg) - Pseudo R-squared:")
print(f"R-squared: {model2_def.prsquared:.4f}")

# ==============================================================================
# HYPOTHESIS 3: Model Refinement (Elite Bonus)
# ==============================================================================

# X3 = ContenderScore (Linear) + EliteBonus (Non-linear)
X3 = df_modern[['ContenderScore', 'EliteBonus']].astype(float)
X3 = sm.add_constant(X3)
model3 = sm.Logit(Y, X3).fit(disp=False)

print("\n## 3. Model Refinement: ContenderScore + EliteBonus")
print(model3.summary().as_text())

# Final Accuracy Check on Refined Model
predictions_proba = model3.predict(X3)
predictions_binary = (predictions_proba >= 0.5).astype(int)
accuracy = accuracy_score(Y, predictions_binary)
print(f"\nFinal Refined Model Accuracy: {accuracy:.4f}")

---
✅ Data Prep Complete. Running All Models:

## 1. Original Hypothesis: ContenderScore Validity
                           Logit Regression Results                           
Dep. Variable:            IsContender   No. Observations:                  420
Model:                          Logit   Df Residuals:                      418
Method:                           MLE   Df Model:                            1
Date:                Tue, 25 Nov 2025   Pseudo R-squ.:                  0.3941
Time:                        14:39:06   Log-Likelihood:                -95.285
converged:                       True   LL-Null:                       -157.27
Covariance Type:            nonrobust   LLR p-value:                 8.569e-29
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const            -10.0687      1.223     -8.231      0.000     -12.466      -7.671
ContenderScore     1.

In [14]:
import numpy as np
from sklearn.metrics import recall_score, precision_score, accuracy_score

# Assuming X3, model3, and Y are defined from the final refinement model

# Make probability predictions
predictions_proba = model3.predict(X3)

# 1. ADJUST THE THRESHOLD HERE: Use 0.3 instead of 0.5
predictions_binary_low_thresh = (predictions_proba >= 0.3).astype(int)

# Calculate new Recall and Precision
new_recall = recall_score(Y, predictions_binary_low_thresh)
new_precision = precision_score(Y, predictions_binary_low_thresh)
new_accuracy = accuracy_score(Y, predictions_binary_low_thresh)

print("\n---")
print("🎯 Results with 30% Probability Threshold:")
print(f"New Overall Accuracy: {new_accuracy:.4f}")
print(f"New Model Precision: {new_precision:.4f}")
print(f"New Model Recall: {new_recall:.4f}")


---
🎯 Results with 30% Probability Threshold:
New Overall Accuracy: 0.8833
New Model Precision: 0.5224
New Model Recall: 0.6731


In [15]:
# Assuming df_modern and Y are defined

# X_unconstrained = Rel_ORtg + Rel_DRtg (Unconstrained model)
X_unconstrained = df_modern[['Rel_ORtg', 'Rel_DRtg']].astype(float)
X_unconstrained = sm.add_constant(X_unconstrained)

model_unconstrained = sm.Logit(Y, X_unconstrained).fit(disp=False)

# Calculate accuracy scores for the new model
predictions_proba_unc = model_unconstrained.predict(X_unconstrained)
predictions_binary_unc = (predictions_proba_unc >= 0.5).astype(int)

accuracy_unc = accuracy_score(Y, predictions_binary_unc)
recall_unc = recall_score(Y, predictions_binary_unc)
precision_unc = precision_score(Y, predictions_binary_unc)

print("\n---")
print("🔬 Results for Unconstrained Two-Way Model (Rel_ORtg + Rel_DRtg):")
print(f"Pseudo R-squared: {model_unconstrained.prsquared:.4f}")
print(f"Overall Accuracy: {accuracy_unc:.4f}")
print(f"Model Recall: {recall_unc:.4f}")
print(f"Model Precision: {precision_unc:.4f}")


---
🔬 Results for Unconstrained Two-Way Model (Rel_ORtg + Rel_DRtg):
Pseudo R-squared: 0.4277
Overall Accuracy: 0.8976
Model Recall: 0.4423
Model Precision: 0.6216


In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import pandas as pd

# Assuming df_modern and IsContender are available from previous steps.

# --- 1. Define Features (X) and Target (Y) ---
# Use the continuous relative ratings, allowing the forest to learn non-linear thresholds.
# ContenderScore is also included as a synthetic feature.
feature_cols = ['Rel_ORtg', 'Rel_DRtg', 'ContenderScore']

X = df_modern[feature_cols].astype(float)
Y = df_modern['IsContender']

# --- 2. Split Data (Training and Testing) ---
# We use a standard 70/30 split. Setting a random_state ensures reproducible results.
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.30, random_state=42, stratify=Y
)

# --- 3. Train the Random Forest Model ---
# RandomForestClassifier is used for binary classification.
rf_model = RandomForestClassifier(
    n_estimators=100,  # Number of trees in the forest
    max_depth=5,       # Restrict tree depth to prevent severe overfitting
    random_state=42
)
rf_model.fit(X_train, Y_train)

# --- 4. Predict and Evaluate ---
Y_pred = rf_model.predict(X_test)

# Calculate key metrics
rf_accuracy = accuracy_score(Y_test, Y_pred)
rf_precision = precision_score(Y_test, Y_pred)
rf_recall = recall_score(Y_test, Y_pred)
rf_conf_matrix = confusion_matrix(Y_test, Y_pred)


print("---")
print("## 🌳 Random Forest Model Performance (70/30 Split)")
print(f"Overall Accuracy: {rf_accuracy:.4f}")
print(f"Model Precision (Contenders): {rf_precision:.4f}")
print(f"Model Recall (Contenders): {rf_recall:.4f}")

print("\nConfusion Matrix:")
print(pd.DataFrame(rf_conf_matrix, 
                   columns=['Predicted Non-Contender (0)', 'Predicted Contender (1)'], 
                   index=['Actual Non-Contender (0)', 'Actual Contender (1)']).to_markdown())

---
## 🌳 Random Forest Model Performance (70/30 Split)
Overall Accuracy: 0.8571
Model Precision (Contenders): 0.4000
Model Recall (Contenders): 0.2500

Confusion Matrix:
|                          |   Predicted Non-Contender (0) |   Predicted Contender (1) |
|:-------------------------|------------------------------:|--------------------------:|
| Actual Non-Contender (0) |                           104 |                         6 |
| Actual Contender (1)     |                            12 |                         4 |


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import numpy as np

# Assuming X, Y are defined from the previous steps.
# Features: Rel_ORtg, Rel_DRtg, ContenderScore

# --- Split Data (Re-run) ---
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.30, random_state=42, stratify=Y
)

# --- Train the Random Forest Model with Class Weighting ---
rf_model_balanced = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42,
    # 💥 FIX: ADD THE CLASS WEIGHT PARAMETER 💥
    class_weight='balanced' 
)
rf_model_balanced.fit(X_train, Y_train)

# --- Predict and Evaluate ---
Y_pred_balanced = rf_model_balanced.predict(X_test)

# Calculate key metrics
rf_accuracy_b = accuracy_score(Y_test, Y_pred_balanced)
rf_precision_b = precision_score(Y_test, Y_pred_balanced)
rf_recall_b = recall_score(Y_test, Y_pred_balanced)
rf_conf_matrix_b = confusion_matrix(Y_test, Y_pred_balanced)


print("---")
print("## 🌳 Random Forest Model Performance (BALANCED)")
print(f"Overall Accuracy: {rf_accuracy_b:.4f}")
print(f"Model Precision (Contenders): {rf_precision_b:.4f}")
print(f"Model Recall (Contenders): {rf_recall_b:.4f}")

print("\nConfusion Matrix (Balanced Model):")
print(pd.DataFrame(rf_conf_matrix_b, 
                   columns=['Predicted Non-Contender (0)', 'Predicted Contender (1)'], 
                   index=['Actual Non-Contender (0)', 'Actual Contender (1)']).to_markdown())

---
## 🌳 Random Forest Model Performance (BALANCED)
Overall Accuracy: 0.8730
Model Precision (Contenders): 0.5000
Model Recall (Contenders): 0.6250

Confusion Matrix (Balanced Model):
|                          |   Predicted Non-Contender (0) |   Predicted Contender (1) |
|:-------------------------|------------------------------:|--------------------------:|
| Actual Non-Contender (0) |                           100 |                        10 |
| Actual Contender (1)     |                             6 |                        10 |
